## Test with small data

In [1]:
resume = 'data/checkpoints/0906_214036/model_best.pth'

In [2]:
import json
import torch
import argparse
from tqdm import tqdm
from tabulate import tabulate

import model.loss as module_loss
import model.model as module_arch
import model.metric as module_metric
from parse_config import ConfigParser
import data_loader.data_loaders as module_data
from utils.span2json import span2json
from utils.conll2span import conll2span
from utils.correcting_labels import fix_labels

PAD = '<pad>'

In [3]:
from utils.correcting_labels import fix_labels, remove_incorrect_tag
def get_dict_prediction(tokens, preds, attention_mask, ids2tag):
    temp_preds=[]
    for index in range(len(preds)):    
        if attention_mask[index]==1:
            Ptag = ids2tag.get(preds[index].item())
            temp_preds.append(Ptag)
            
    temp_preds = remove_incorrect_tag(temp_preds, "BIOES")
    temp_preds = fix_labels(temp_preds, "BIOES")    
    temp_preds = conll2span(temp_preds)
    temp_preds = span2json(tokens, temp_preds)   
    return temp_preds

In [4]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
args.add_argument('-r', '--resume', default=f"{resume}", type=str, help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str, help='indices of GPUs to enable (default: all)')
args.add_argument('-f', '--file', default=None, type=str, help='Error')

config = ConfigParser.from_args(args)
logger = config.get_logger('test')

data_loader = config.init_obj('data_loader', module_data)
test_data_loader = data_loader.get_test()

# build model architecturea
model = config.init_obj('arch', module_arch)
logger.info(model)

# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']
if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
layers_train = config._config['trainer']['layers_train']

# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

total_loss = 0.0
total_metrics = torch.zeros(len(metric_fns))

NNEModel(
  (lm): CamembertModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [ ]:
from tqdm import tqdm
results = []
with torch.no_grad():
    for batch_idx, instance in tqdm(enumerate(test_data_loader)):
        input_ids = torch.tensor(instance['input_ids']).to(device)
        attention_mask = torch.tensor(instance['attention_mask']).to(device)
        batch_size = input_ids.shape[0]
        output = model(input_ids, attention_mask)
        
        loss = 0
        nested_lm_conll_ids = {l:None for l in range(len(layers_train))}
        for index, layer in enumerate(layers_train):
            temp_nested_lm_conll_ids = torch.tensor(instance['nested_lm_conll_ids'][layer])
            temp_nested_lm_conll_ids = temp_nested_lm_conll_ids.to(device)
            nested_lm_conll_ids[index]=temp_nested_lm_conll_ids
            loss+=criterion(output[index], temp_nested_lm_conll_ids)
        total_loss += loss.item() * batch_size
        predictions = {x:[] for x in range(batch_size)}
        lm_entities = {x:[] for x in range(batch_size)}
        
        for sent_ids in range(batch_size):
            for layer in range(len(output)):
                predictions[sent_ids].append(output[layer][sent_ids].argmax(axis=0))
                lm_entities[sent_ids].append(nested_lm_conll_ids[layer][sent_ids])
        for sent_ids in range(batch_size):
            tokens = instance['lm_tokens'][sent_ids]
            tokens = [w for w in tokens if w!=PAD]
            preds = []
            for index in range(len(layers_train)):
                preds+=get_dict_prediction(
                        tokens, 
                        predictions[sent_ids][index], 
                        attention_mask[sent_ids], 
                        data_loader.ids2tag)
            entities_labels = []
            for index in range(len(layers_train)):
                entities_labels+=get_dict_prediction(
                    tokens, 
                    lm_entities[sent_ids][index], 
                    attention_mask[sent_ids], 
                    data_loader.ids2tag)
            results.append({
                'sentence_id': instance['sentence_id'][sent_ids],
                'tokens': tokens,
                'entities': entities_labels,
                'predictions':preds})
            for i, metric in enumerate(metric_fns):
                total_metrics[i] += metric( 
                        output, nested_lm_conll_ids, attention_mask, 
                        data_loader.boundary_type, info=False, ids2tag=data_loader.ids2tag
            ) * batch_size     

0it [00:00, ?it/s]/home/weerayut/anaconda3/envs/workspace/lib/python3.6/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/weerayut/anaconda3/envs/workspace/lib/python3.6/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [00:24,  8.26s/it]

In [ ]:
count = 0
output = []
n_limit = 1
for ids, item in enumerate(results[:n_limit]):
    print('\n',"".join(item["tokens"]),'\n')
    sid = item['sentence_id']
    entities=item['entities']
    predictions=item['predictions']

    for pred in predictions:
        tag = "/" if pred in entities else "X"
        out = [f"SID:{sid}", tag, ''.join(pred['text']), pred['entity_type']]
        print(out)

#     for en in entities:
#         tag = "N" if en not in predictions else "-"
#         out = [f"SID:{sid}", tag, ''.join(en['text']), en['entity_type']]
#         print(out)

In [ ]:
## Can input both BIESO and BIO
from model.eval import ClassEvaluator
results_eval, conll_results = ClassEvaluator()(results)